In [ ]:
!pip install transformers sentencepiece torch nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
ENTITIES = [
    'silver', 'silver prices', 'silver price', 'silver futures',
    'spot silver', 'silver market', 'silver demand', 'silver supply',
    'silver production', 'silver mining', 'silver bullion', 'silver bars',
    'silver coins', 'silver reserves', 'silver exports', 'silver imports',
    'silver ETF', 'precious metals', 'London Bullion Market', 'COMEX silver'
]


In [ ]:
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

# Load FinBERT-tone
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
model.eval()

# Label map
id2label = {0: "neutral", 1: "positive", 2: "negative"}


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        label_id = torch.argmax(logits, dim=1).item()
        return id2label[label_id]


In [ ]:
import pandas as pd
import re

df = pd.read_csv('articles_filtered.csv')
results = []

total_rows = len(df)

for idx, row in df.iterrows():
    article_text = row['text']
    date = row['Date']

    sentences = sent_tokenize(article_text)

    for sent in sentences:
        for entity in ENTITIES:
            # Search for entity match in sentence
            pattern = re.compile(rf'\b{re.escape(entity)}\b', flags=re.IGNORECASE)
            for match in pattern.finditer(sent):
                sentiment = predict_sentiment(sent)
                start = article_text.find(match.group())
                end = start + len(match.group())

                results.append({
                    'date': date,
                    'text': article_text,
                    'entity': match.group(),
                    'start': start,
                    'end': end,
                    'sentiment': sentiment
                })

    # Print % completed
    if idx % max(1, total_rows // 20) == 0:
        percent = (idx + 1) / total_rows * 100
        print(f"Progress: {percent:.1f}%")

print("Completed!")


<ipython-input-5-03255c7789cf>:4: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('articles_filtered.csv')


Progress: 0.0%
Progress: 5.0%
Progress: 10.0%
Progress: 15.0%
Progress: 20.0%
Progress: 25.0%
Progress: 30.0%
Progress: 34.9%
Progress: 39.9%
Progress: 44.9%
Progress: 49.9%
Progress: 54.9%
Progress: 59.9%
Progress: 64.9%
Progress: 69.9%
Progress: 74.9%
Progress: 79.9%
Progress: 84.8%
Progress: 89.8%
Progress: 94.8%
Progress: 99.8%
Completed!


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load or prepare data
df = pd.DataFrame(results)

# Map sentiment to numerical scores
sentiment_map = {'positive': 1, 'neutral': 0, 'negative': -1}
df['sentiment_score'] = df['sentiment'].map(sentiment_map)

# Drop time component and any bad dates
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
df = df.dropna(subset=['date'])

# Aggregate by date (not datetime)
daily_sentiment = df.groupby('date')['sentiment_score'].sum().reset_index()

# Normalize
scaler = MinMaxScaler()
daily_sentiment['normalized_score'] = scaler.fit_transform(daily_sentiment[['sentiment_score']])


In [ ]:
daily_sentiment.to_csv('daily_sequence_sentiment.csv', index=False)
